In [3]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 30757899388 (28.65 GB)
DFS Remaining: 30228914176 (28.15 GB)
DFS Used: 528985212 (504.48 MB)
DFS Used%: 1.72%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.20.0.3:9866 (project-4-p4-raffanti-dn-1.project-4-p4-raffanti_default)
Hostname: a5564c2141ff
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 259821821 (247.79 MB)
Non DFS Used: 10430013187 (9.71 GB)
DFS Remaining: 15114440704 (14.08 GB)
DFS Used%: 1.01%
DFS 

In [2]:
import time
time.sleep(30)

In [4]:
import requests
fileStat = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS&offset=0&length=200&noredirect=true")
file_status = fileStat.json()
file_length = file_status['FileStatus']['length']
block_size = file_status['FileStatus']['blockSize']
total_blocks = -(-file_length // block_size)  # Ceil division to get total blocks

container_block_count = {'lost': 0}

# Loop through each block to extract container IDs
for block_id in range(total_blocks):
    block_url = f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={block_id * block_size}&noredirect=true"

    response = requests.get(block_url)
    location = response.json()
    if response.status_code == 403:
        container_block_count['lost'] += 1
    else:
        block_location = location['Location']
        container_id = block_location.split('/')[2]
        if container_id in container_block_count:
            container_block_count[container_id] += 1
        else:
            container_block_count[container_id] = 1

In [5]:
#q9
container_block_count

{'lost': 88, 'a5564c2141ff:9864': 79}

In [6]:
total_single_family_count = 0

for container_id, count in container_block_count.items():
    if count > 0 and container_id != 'lost':
        for block_id in range(count):
            block_url = f"http://boss:9870/webhdfs/v1/single.csv?offset={block_id * block_size}&length={block_size}&op=OPEN"
            response = requests.get(block_url)
            block_content = response.content.decode('utf-8')

            total_single_family_count += block_content.count("Single Family")

In [7]:
#q10
total_single_family_count

101076